In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

# Load the dataset
data = pd.read_csv('upskilling_dataset.csv')

# Preprocessing
# Encode categorical features (Job Title, Company, Industry)
categorical_features = ['Job Title', 'Company', 'Industry']
encoder = OneHotEncoder(sparse_output=False)
categorical_encoded = encoder.fit_transform(data[categorical_features])

# Encode target variables (Core Skills, Company-Specific Skills)
# Convert skills lists into sets for consistency
data['Core Skills'] = data['Core Skills'].str.split(', ').apply(set)
data['Company-Specific Skills'] = data['Company-Specific Skills'].str.split(', ').apply(set)

mlb_core = MultiLabelBinarizer()
mlb_company = MultiLabelBinarizer()

y_core = mlb_core.fit_transform(data['Core Skills'])
y_company = mlb_company.fit_transform(data['Company-Specific Skills'])

# Combine target variables for multi-output classification
y = pd.concat([pd.DataFrame(y_core), pd.DataFrame(y_company)], axis=1)

# Combine all input features
X = pd.DataFrame(categorical_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Training
knn_model = KNeighborsClassifier()
multioutput_knn_model = MultiOutputClassifier(knn_model)
multioutput_knn_model.fit(X_train, y_train)

# Model Evaluation
y_pred = multioutput_knn_model.predict(X_test)
print("Core Skills Classification Report:")
print(classification_report(y_test.iloc[:, :len(mlb_core.classes_)], y_pred[:, :len(mlb_core.classes_)], target_names=mlb_core.classes_))

print("Company-Specific Skills Classification Report:")
print(classification_report(y_test.iloc[:, len(mlb_core.classes_):], y_pred[:, len(mlb_core.classes_):], target_names=mlb_company.classes_))



Core Skills Classification Report:
                         precision    recall  f1-score   support

        API Development       1.00      1.00      1.00       393
                    CSS       1.00      1.00      1.00       397
    Campaign Management       1.00      1.00      1.00       413
        Data Structures       1.00      1.00      1.00       410
          Data analysis       1.00      1.00      1.00       387
    Database Management       1.00      1.00      1.00       393
      Digital Marketing       1.00      1.00      1.00       413
                  Excel       1.00      1.00      1.00       387
                   HTML       1.00      1.00      1.00       397
             JavaScript       1.00      1.00      1.00       397
                    OOP       1.00      1.00      1.00       410
   Programming (Python)       1.00      1.00      1.00       410
                    SEO       1.00      1.00      1.00       413
                    SQL       1.00      1.00      1.00

In [39]:
# Prediction using user input
def get_user_input():
    print("Enter the following details for prediction:")
    job_title = input("Job Title: ")
    company = input("Company: ")
    industry = input("Industry: ")

    # Encode user input
    user_data = pd.DataFrame([[job_title, company, industry]], columns=categorical_features)
    user_encoded = encoder.transform(user_data)
    return user_encoded

user_input = get_user_input()
user_prediction = multioutput_knn_model.predict(user_input)

# Decode predictions
core_skills_pred = mlb_core.inverse_transform(user_prediction[:, :len(mlb_core.classes_)])
company_skills_pred = mlb_company.inverse_transform(user_prediction[:, len(mlb_core.classes_):])

print("Predicted Core Skills:", core_skills_pred[0])
print("Predicted Company-Specific Skills:", company_skills_pred[0])

Enter the following details for prediction:
Predicted Core Skills: ('CSS', 'HTML', 'JavaScript')
Predicted Company-Specific Skills: ('GCP',)


In [26]:
import gradio as gr

In [42]:
def predict(job,com,inds):
    user_data = pd.DataFrame([[job, com, inds]], columns=categorical_features)
    user_encoded = encoder.transform(user_data)
    user_prediction = multioutput_knn_model.predict(user_encoded)
    core_skills_pred = mlb_core.inverse_transform(user_prediction[:, :len(mlb_core.classes_)])
    company_skills_pred = mlb_company.inverse_transform(user_prediction[:, len(mlb_core.classes_):])
    return list(core_skills_pred[0]),list(company_skills_pred[0])


In [43]:
iface = gr.Interface( 
                 fn=predict,
                 inputs=[gr.Textbox(label="Job title"),gr.Textbox(label="Company"),gr.Textbox(label="Industry")],
                 outputs=[gr.CheckboxGroup(label="Predicted Core Skills", choices=list(mlb_core.classes_)), gr.CheckboxGroup(label="Predicted Company-Specific Skills", choices=list(mlb_company.classes_))],
                 title="Upskilling Skills Prediction",
                 description="Enter your job title, company, and industry to predict relevant skills." ) # Launch the interface

In [45]:
iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [46]:
iface.close()

Closing server running on port: 7862
